## Surface roughness

This example shows how to:
   - use materials with predefined surface roughness
   - modify roughness

![plotoptix ray_tracing_output](https://plotoptix.rnd.team/images/material_roughness.jpg "This notebook output")

All materials in PlotOptiX have the roughness parameterized with ``surface_roughness``, the sigma of the gaussian distribution of microfacet slopes. Oren-Nayar reflectance model is used for *diffuse* materials, reduced to Lambertian reflectance if sigma is set to 0 (which is default and also faster shading). Reflective and transmissive materials use smearing of the surface normal according to the configured slope distribution.

Make some data for a simple scene first:

In [1]:
import numpy as np

rx = (-20, 20)
rz = (-20, 20)
n = 100

x = np.linspace(rx[0], rx[1], n)
z = np.linspace(rz[0], rz[1], n)

X, Z = np.meshgrid(x, z)

# positions of blocks
data = np.stack((X.flatten(), np.zeros(n*n), Z.flatten())).T
# XZ sizes
size_u = 0.96 * (rx[1] - rx[0]) / (n - 1)
size_w = 0.96 * (rz[1] - rz[0]) / (n - 1)

Setup the raytracer using Tkinter GUI as the output target:

In [2]:
from plotoptix import TkOptiX

optix = TkOptiX()
optix.set_param(min_accumulation_step=4,     # set more accumulation frames to get rid of the noise
                max_accumulation_frames=512,
                light_shading="Hard")        # use ligth shading best for caustics

optix.set_uint("path_seg_range", 5, 10)      # more path segments to allow multiple refractions

Only *diffuse* material is available by default. Other materials need to be configured before using.

In [3]:
from plotoptix.materials import m_clear_glass, m_matt_glass, m_diffuse, m_matt_diffuse, m_plastic, m_matt_plastic, m_mirror, m_metalic

m_matt_glass_2 = m_matt_glass.copy()
m_matt_diffuse_2 = m_matt_diffuse.copy()
m_matt_plastic_2 = m_matt_plastic.copy()
m_metalic_2 = m_metalic.copy()

optix.setup_material("matt_diffuse", m_matt_diffuse)
optix.setup_material("matt_diffuse_2", m_matt_diffuse_2)
optix.setup_material("glass", m_clear_glass)
optix.setup_material("matt_glass", m_matt_glass)
optix.setup_material("matt_glass_2", m_matt_glass_2)
optix.setup_material("plastic", m_plastic)
optix.setup_material("matt_plastic", m_matt_plastic)
optix.setup_material("matt_plastic_2", m_matt_plastic_2)
optix.setup_material("mirror", m_mirror)
optix.setup_material("metalic", m_metalic)
optix.setup_material("metalic_2", m_metalic_2)

Add objects to the scene.

In [4]:
optix.set_data("blocks", pos=data,
               c=0.02 + 0.45*np.random.randint(3, size=data.shape[0]),
               u=[size_u, 0, 0], v=[0, -1, 0], w=[0, 0, size_w],
               geom="Parallelepipeds")

optix.set_data("s_diffuse", pos=[-4.2, 0, -4.5], u=[2.5, 0, 0], v=[0, 2.5, 0], w=[0, 0, 2.5], c=0.95, mat="diffuse", geom="Parallelepipeds")
optix.set_data("s_matt_diffuse", pos=[-1.2, 0, -4.5], u=[2.5, 0, 0], v=[0, 2.5, 0], w=[0, 0, 2.5], c=0.95, mat="matt_diffuse", geom="Parallelepipeds")
optix.set_data("s_matt_diffuse_2", pos=[2, 0, -4.5], u=[2.5, 0, 0], v=[0, 2.5, 0], c=0.95, mat="matt_diffuse_2", geom="Parallelepipeds")

optix.set_data("s_glass", pos=[-3.1, 1.5, 0], r=1.5, c=10, mat="glass")
optix.set_data("s_matt_glass", pos=[0, 1.5, 0], r=1.5, c=10, mat="matt_glass")
optix.set_data("s_matt_glass_2", pos=[3.1, 1.5, 0], r=1.5, c=10, mat="matt_glass_2")

optix.set_data("s_plastic", pos=[-3.1, 1.5, 3.1], r=1.5, c=0.95, mat="plastic")
optix.set_data("s_matt_plastic", pos=[0, 1.5, 3.1], r=1.5, c=0.95, mat="matt_plastic")
optix.set_data("s_matt_plastic_2", pos=[3.1, 1.5, 3.1], r=1.5, c=0.95, mat="matt_plastic_2")

optix.set_data("s_mirror", pos=[-3.1, 1.5, 6.2], r=1.5, c=0.9, mat="mirror")
optix.set_data("s_metalic", pos=[0, 1.5, 6.2], r=1.5, c=0.9, mat="metalic")
optix.set_data("s_metalic_2", pos=[3.1, 1.5, 6.2], r=1.5, c=0.9, mat="metalic_2")

Setup a good point of view, set background and lights.

In [5]:
optix.setup_camera("cam1", cam_type="DoF",
                   eye=[-6.5, 13, 1.55], target=[0, 0, 1.55], up=[0.9, 0.4, 0],
                   aperture_radius=0.01, fov=45, focal_scale=0.8)

optix.setup_light("light1", pos=[5, 8, 7], color=[3, 3, 3], radius=1.9)
optix.setup_light("light2", pos=[-6, 8, -5], color=[8, 10, 12], radius=1.3)

exposure = 1.1; gamma = 2.2 
optix.set_float("tonemap_exposure", exposure)
optix.set_float("tonemap_igamma", 1 / gamma)
optix.add_postproc("Gamma")        # apply gamma correction postprocessing stage, or
#optix.setup_denoiser(blend=0.2)   # use AI denoiser (exposure and gamma are applied as well)

optix.set_background_mode("TextureEnvironment")
optix.set_background("data/starmap_4k.jpg", gamma=gamma)

Open the GUI.

In [6]:
optix.start()

Modify roughness of the surfaces in the third row. The effect is obvious except for the diffuse material, but note how much light leaks between cubes depending on the roughness.

In [7]:
m_matt_diffuse_2["VarFloat"]["surface_roughness"] = 10    # more than the default 1.0
optix.update_material("matt_diffuse_2", m_matt_diffuse_2)

m_matt_glass_2["VarFloat"]["surface_roughness"] = 0.002   # less than the default 0.2
optix.update_material("matt_glass_2", m_matt_glass_2)

m_matt_plastic_2["VarFloat"]["surface_roughness"] = 0.01  # more than the default 0.001
optix.update_material("matt_plastic_2", m_matt_plastic_2)

m_metalic_2["VarFloat"]["surface_roughness"] = 0.02       # more than the default 0.002
optix.update_material("metalic_2", m_metalic_2, refresh=True)

Close GUI window, release resources.

In [8]:
optix.close()